In [2]:
import pickle
import os
from pathlib import Path
from algo_4 import *
from utils import *


In [3]:
data_path = Path(os.getcwd()).parent
path = data_path / 'stock price data'
ticker_all = pd.read_csv(path/"finer_data_complete_stock.csv")["Symbol"].unique()

In [11]:
a = range(96, len(ticker_all))
ticker_all[96:a[-1]+5]

array(['AMSC', 'AMSF', 'AMSWA', ..., 'ZIONP', 'ZUMZ', 'ZYXI'],
      dtype=object)

In [3]:
# List all files in the directory that start with "output "
directory = os.getcwd()
pickle_files = [f for f in os.listdir(directory) if f.startswith("output")]

# Initialize a dictionary to store the contents of each pickle file
pickle_contents = []

# Loop through each file, read its content, and store it in the dictionary
for file_name in pickle_files:
    file_path = os.path.join(directory, file_name)
    with open(file_path, 'rb') as file:
        pickle_contents.extend(pickle.load(file))

# pickle_contents.keys()
pickle_contents.sort(key=lambda x: x[0][1], reverse=True)
all_tickers = [i[0][0]["ticker"] for i in pickle_contents]
len(all_tickers)

16

In [4]:
pickle_contents

[[({'sma_length': 150,
    'rsi_period': 8,
    'long_range': 40,
    'oversell': 35,
    'overbuy': 65,
    'midline_buy': 50,
    'midline_sell': 50,
    'day_max': 11,
    'percent_trend': 0.5,
    'percent_trend2': 0.8,
    'ticker': 'ADBE'},
   1.8258005052953918,
   (3.3188603353733286, 0.12118531623175585, 0.7866666666666666)),
  ({'sma_length': 150,
    'rsi_period': 8,
    'long_range': 40,
    'oversell': 35,
    'overbuy': 70,
    'midline_buy': 50,
    'midline_sell': 50,
    'day_max': 11,
    'percent_trend': 0.5,
    'percent_trend2': 0.8,
    'ticker': 'ADBE'},
   1.7090213707383404,
   (3.0813751256199557, 0.12118531623175585, 0.7945205479452055)),
  ({'sma_length': 200,
    'rsi_period': 8,
    'long_range': 20,
    'oversell': 35,
    'overbuy': 70,
    'midline_buy': 50,
    'midline_sell': 50,
    'day_max': 11,
    'percent_trend': 0.5,
    'percent_trend2': 0.8,
    'ticker': 'ADBE'},
   1.6960107003911467,
   (3.0694971757812883, 0.12118531623175585, 0.766233766

In [138]:
finer_pick = []
a = []
b=[]
c=[]
d=[]
finer_finer_pick = []
for i in pickle_contents:
    for j in range(len(i)):
        stock = algo2(**i[j][0])
        # eval(stock, opt=False)
        maximum_drawdown, ROI, win_loss_ratio = eval(stock,ticker=i[j][0]["ticker"], variables=i[j][0], opt=True)
        buy_orders = []
        sell_orders = []
        current_order = None
        money = 100000
        short = None
        long = None

        for index, row in stock.iterrows():
            if row['order'] > 0:
                if current_order is None and short is None:
                    current_order = {"Initial Money": money, 'Buy Date': row['date'], 'Buy Price': stock.loc[index -
                                                                                                            1, 'close'], "Shares Purchased": money/stock.loc[index - 1, 'close'], "long/short": "long"}
                    long = True
                elif current_order is not None and short is not None:
                    current_order['Sell Date'] = row['date']
                    current_order['Sell Price'] = row['close']
                    current_order["Price Change"] = current_order["Sell Price"] - \
                        current_order["Buy Price"]
                    current_order["Profit"] = current_order["Price Change"] * \
                        current_order["Shares Purchased"] * -1
                    current_order["Money After Order"] = current_order["Shares Purchased"] * \
                        current_order['Buy Price'] + current_order["Profit"]
                    money = current_order["Money After Order"]
                    sell_orders.append(current_order)
                    short = None
                    current_order = {"Initial Money": money, 'Buy Date': row['date'], 'Buy Price': row[
                        'close'], "Shares Purchased": money/row['close'], "long/short": "long"}
                    long = True
                else:
                    continue
            elif row['order'] < 0:
                if current_order is None and long is None:
                    current_order = {"Initial Money": money, 'Buy Date': row['date'], 'Buy Price': stock.loc[index -
                                                                                                            1, 'close'], "Shares Purchased": money/stock.loc[index - 1, 'close'], "long/short": "short"}
                    short = True
                elif current_order is not None and long is not None:
                    current_order['Sell Date'] = row['date']
                    current_order['Sell Price'] = row['close']
                    current_order["Price Change"] = current_order["Sell Price"] - \
                        current_order["Buy Price"]
                    current_order["Profit"] = current_order["Price Change"] * \
                        current_order["Shares Purchased"]
                    current_order["Money After Order"] = current_order["Shares Purchased"] * \
                        current_order['Buy Price'] + current_order["Profit"]
                    money = current_order["Money After Order"]
                    sell_orders.append(current_order)
                    long = None
                    current_order = {"Initial Money": money, 'Buy Date': row['date'], 'Buy Price': row[
                        'close'], "Shares Purchased": money/row['close'], "long/short": "short"}
                    short = True
                else:
                    continue
            elif row['order'] == 0:
                if current_order is not None:
                    current_order['Sell Date'] = row['date']
                    current_order['Sell Price'] = row['close']
                    current_order["Price Change"] = current_order["Sell Price"] - \
                        current_order["Buy Price"]
                    if current_order['long/short'] == "short":
                        current_order["Profit"] = current_order["Price Change"] * \
                            current_order["Shares Purchased"] * -1
                    elif current_order['long/short'] == "long":
                        current_order["Profit"] = current_order["Price Change"] * \
                            current_order["Shares Purchased"]
                    current_order["Money After Order"] = current_order["Shares Purchased"] * \
                        current_order['Buy Price'] + current_order["Profit"]
                    money = current_order["Money After Order"]
                    sell_orders.append(current_order)
                    current_order = None
                    long = None
                    short = None

        # Create a new DataFrame for buy and sell orders
        orders_df = pd.DataFrame(buy_orders + sell_orders)
        if len(orders_df) >= 20:
            if i[j][0]["ticker"] not in a:
                a.append(i[j][0]["ticker"])
            if ROI >= 2 and maximum_drawdown <= 0.12 and win_loss_ratio >= 0.75:
                if i[j][0]["ticker"] not in finer_pick:
                    finer_pick.append(i[j][0]["ticker"])
            if ROI >= 2 and maximum_drawdown <= 0.10 and win_loss_ratio >= 0.78:
                if i[j][0]["ticker"] not in b:
                    b.append(i[j][0]["ticker"])
            if ROI >= 2 and maximum_drawdown <= 0.15 and win_loss_ratio >= 0.8:
                if i[j][0]["ticker"] not in c:
                    c.append(i[j][0]["ticker"])
            if ROI >= 3 and maximum_drawdown <= 0.15 and win_loss_ratio >= 0.72:
                if i[j][0]["ticker"] not in c:
                    d.append(i[j][0]["ticker"])
            if ROI >=5 or maximum_drawdown <= 0.06 or win_loss_ratio >= 0.85:
                if i[j][0]["ticker"] not in finer_finer_pick:
                    finer_finer_pick.append(i[j][0]["ticker"])

In [158]:
summary_dict = {}
summary_dict["finer"] = finer_pick
summary_dict["finer finer pick"] = finer_finer_pick
summary_dict["a"] = a
summary_dict["b"] = b
summary_dict["c"] = c
summary_dict["d"] = d
with open("final_selection", "wb") as fp:   #Pickling
    pickle.dump(summary_dict, fp)
summary_dict

{'finer': {'AAL',
  'ABG',
  'ALK',
  'ALV',
  'AMEH',
  'AMOT',
  'AN',
  'AORT',
  'APEI',
  'APT',
  'ASGN',
  'ASUR',
  'ATGE',
  'AVAV',
  'AYI',
  'BH',
  'CTRA',
  'CVBF',
  'CVCO',
  'DAL',
  'DALN',
  'DB',
  'DGII',
  'DHI',
  'EAT',
  'EBS',
  'EGO',
  'EPAC',
  'ERJ',
  'EZPW',
  'FARO',
  'FCF',
  'FITB',
  'FSTR',
  'FWRD',
  'GBLI',
  'GLNG',
  'GPI',
  'HCCI',
  'HELE',
  'HUBG',
  'HZO',
  'IBN',
  'ICLR',
  'IPGP',
  'IRDM',
  'ITGR',
  'ITRI',
  'IVAC',
  'JHX',
  'KVHI',
  'LDOS',
  'LPSN',
  'LYTS',
  'MDC',
  'MET',
  'MLR',
  'NATI',
  'NOG',
  'OLED',
  'OLN',
  'OSPN',
  'OVLY',
  'PAA',
  'PAG',
  'PDFS',
  'PDS',
  'PEGA',
  'PHM',
  'PMT',
  'PSO',
  'REX',
  'RILY',
  'SAP',
  'SBS',
  'SCX',
  'SHO',
  'SMBK',
  'SMFG',
  'SNV',
  'SRCL',
  'SSL',
  'SSRM',
  'SWBI',
  'TAC',
  'TCI',
  'TCOM',
  'TIMB',
  'TOL',
  'TPC',
  'TR',
  'TRC',
  'TYG',
  'UFI',
  'UNFI',
  'USPH',
  'UTMD',
  'VGR',
  'WASH',
  'WERN',
  'WEYS',
  'WGO',
  'WWW',
  'ZION',
  'Z

In [139]:
len(set(a))

229

In [152]:
data_path = Path(os.getcwd())
path = data_path / 'stock price data'
finer_pick = set(finer_pick)
ticker_all = pd.read_csv(path/"finer_data_complete_stock.csv")
ticker_finer = ticker_all[ticker_all["Symbol"].isin(finer_pick)]
print(ticker_finer.groupby("Sector")["Sector"].count().sum())
ticker_finer.groupby("Sector")["Sector"].count()

105


Sector
Basic Materials            6
Communication Services     4
Consumer Cyclical         20
Consumer Defensive         6
Energy                     5
Financial Services        16
Healthcare                 7
Industrials               19
Real Estate                3
Technology                17
Utilities                  2
Name: Sector, dtype: int64

In [153]:
ticker_a = ticker_all[ticker_all["Symbol"].isin(a)]
print(ticker_a.groupby("Sector")["Sector"].count().sum())
ticker_a.groupby("Sector")["Sector"].count()

229


Sector
Basic Materials           14
Communication Services     8
Consumer Cyclical         33
Consumer Defensive         9
Energy                    17
Financial Services        27
Healthcare                20
Industrials               45
Real Estate                7
Technology                46
Utilities                  3
Name: Sector, dtype: int64

In [154]:
ticker_b = ticker_all[ticker_all["Symbol"].isin(b)]
print(ticker_b.groupby("Sector")["Sector"].count().sum())
ticker_b.groupby("Sector")["Sector"].count()

41


Sector
Basic Materials           2
Communication Services    1
Consumer Cyclical         8
Consumer Defensive        2
Energy                    1
Financial Services        8
Healthcare                5
Industrials               9
Technology                5
Name: Sector, dtype: int64

In [155]:
ticker_c = ticker_all[ticker_all["Symbol"].isin(c)]
print(ticker_c.groupby("Sector")["Sector"].count().sum())
ticker_c.groupby("Sector")["Sector"].count()

111


Sector
Basic Materials            7
Communication Services     3
Consumer Cyclical         19
Consumer Defensive         4
Energy                     6
Financial Services        15
Healthcare                 8
Industrials               24
Real Estate                3
Technology                20
Utilities                  2
Name: Sector, dtype: int64

In [156]:
ticker_d = ticker_all[ticker_all["Symbol"].isin(d)]
print(ticker_d.groupby("Sector")["Sector"].count().sum())
ticker_d.groupby("Sector")["Sector"].count()

52


Sector
Basic Materials            5
Communication Services     1
Consumer Cyclical          6
Consumer Defensive         5
Energy                     5
Financial Services         5
Healthcare                 4
Industrials                7
Real Estate                1
Technology                13
Name: Sector, dtype: int64

In [157]:
finer_finer_pick = set(finer_finer_pick)
ticker_finer_finer = ticker_all[ticker_all["Symbol"].isin(finer_finer_pick)]
print(ticker_finer_finer.groupby("Sector")["Sector"].count().sum())
ticker_finer_finer.groupby("Sector")["Sector"].count()

93


Sector
Basic Materials            6
Communication Services     3
Consumer Cyclical         12
Consumer Defensive         3
Energy                     7
Financial Services        13
Healthcare                 8
Industrials               21
Real Estate                2
Technology                17
Utilities                  1
Name: Sector, dtype: int64

In [146]:
best_dict_finer = {}
best_dict_finer_finer = {}
best_dict_a = {}
best_dict_b = {}
best_dict_c = {}
best_dict_d = {}
for i in pickle_contents:
    if i[0][0]["ticker"] in finer_pick:
        best_dict_finer[i[0][0]["ticker"]] = i[0][0]
    if i[0][0]["ticker"] in finer_finer_pick:
        best_dict_finer_finer[i[0][0]["ticker"]] = i[0][0]
    if i[0][0]["ticker"] in a:
        best_dict_a[i[0][0]["ticker"]] = i[0][0]
    if i[0][0]["ticker"] in b:
        best_dict_b[i[0][0]["ticker"]] = i[0][0]
    if i[0][0]["ticker"] in c:
        best_dict_c[i[0][0]["ticker"]] = i[0][0]
    if i[0][0]["ticker"] in d:
        best_dict_d[i[0][0]["ticker"]] = i[0][0]

best_dict_finer = pd.DataFrame(best_dict_finer).T
best_dict_finer_finer = pd.DataFrame(best_dict_finer_finer).T
best_dict_finer.describe()

,atr_len,macd_fastLen,macd_slowLen,macd_signalSmooth,macd_peakLen,gain_ratio,loss_ratio,peak2_len,peak3_len,ticker
count,105,105,105,105,105,105.0,105.0,105,105,105
unique,3,4,3,3,1,3.0,3.0,1,2,105
top,10,13,30,6,3,1.0,2.0,20,50,GLNG
freq,57,30,43,41,105,56.0,43.0,105,76,1


In [147]:
best_dict_finer_finer.describe()

,atr_len,macd_fastLen,macd_slowLen,macd_signalSmooth,macd_peakLen,gain_ratio,loss_ratio,peak2_len,peak3_len,ticker
count,93,93,93,93,93,93.0,93.0,93,93,93
unique,3,4,3,3,1,3.0,3.0,1,2,93
top,10,10,34,6,3,1.0,2.5,20,50,EVO
freq,50,30,33,44,93,50.0,37.0,93,64,1


In [148]:
best_dict_a = pd.DataFrame(best_dict_a).T
best_dict_a.describe()

,atr_len,macd_fastLen,macd_slowLen,macd_signalSmooth,macd_peakLen,gain_ratio,loss_ratio,peak2_len,peak3_len,ticker
count,229,229,229,229,229,229.0,229.0,229,229,229
unique,3,4,3,3,1,3.0,3.0,1,2,229
top,10,13,34,8,3,1.0,2.0,20,50,EVO
freq,133,66,83,88,229,120.0,87.0,229,160,1


In [149]:
best_dict_b = pd.DataFrame(best_dict_b).T
best_dict_b.describe()

,atr_len,macd_fastLen,macd_slowLen,macd_signalSmooth,macd_peakLen,gain_ratio,loss_ratio,peak2_len,peak3_len,ticker
count,41,41,41,41,41,41.0,41.0,41,41,41
unique,3,4,3,3,1,3.0,3.0,1,2,41
top,10,13,30,8,3,1.0,2.0,20,50,AMEH
freq,22,12,18,16,41,25.0,22.0,41,32,1


In [150]:
best_dict_c = pd.DataFrame(best_dict_c).T
best_dict_c.describe()

,atr_len,macd_fastLen,macd_slowLen,macd_signalSmooth,macd_peakLen,gain_ratio,loss_ratio,peak2_len,peak3_len,ticker
count,111,111,111,111,111,111.0,111.0,111,111,111
unique,3,4,3,3,1,3.0,3.0,1,2,111
top,10,10,30,6,3,1.0,2.5,20,50,MDGL
freq,60,32,44,48,111,60.0,54.0,111,76,1


In [151]:
best_dict_d = pd.DataFrame(best_dict_d).T
best_dict_d.describe()

,atr_len,macd_fastLen,macd_slowLen,macd_signalSmooth,macd_peakLen,gain_ratio,loss_ratio,peak2_len,peak3_len,ticker
count,52,52,52,52,52,52.0,52.0,52,52,52
unique,3,4,3,3,1,3.0,3.0,1,2,52
top,10,13,34,6,3,1.0,1.5,20,50,FHI
freq,29,14,21,23,52,28.0,25.0,52,34,1


In [132]:
with open("ticker_coarse", "wb") as fp:   #Pickling
    pickle.dump(list(set(a)), fp)
a

['EVO',
 'HWBK',
 'AMEH',
 'MBI',
 'AAL',
 'LPSN',
 'INVE',
 'DGLY',
 'DALN',
 'NG',
 'MITK',
 'TNK',
 'GLNG',
 'USPH',
 'HZO',
 'EBS',
 'UFPI',
 'ICCC',
 'CVI',
 'FOLD',
 'UNFI',
 'AEO',
 'ASGN',
 'MDGL',
 'LYTS',
 'HOV',
 'GEOS',
 'HCCI',
 'TWI',
 'GBLI',
 'AGYS',
 'CVE',
 'WNC',
 'WPRT',
 'DVAX',
 'LPX',
 'PAG',
 'TSRI',
 'MOD',
 'TNP',
 'FRPH',
 'EZPW',
 'NOTV',
 'VIAV',
 'APT',
 'EGAN',
 'RILY',
 'EPAC',
 'WHLM',
 'REX',
 'SBS',
 'LSCC',
 'PRDO',
 'TCX',
 'UFI',
 'SYNA',
 'SSRM',
 'HELE',
 'ITRI',
 'UCTT',
 'NOG',
 'MYRG',
 'IGT',
 'APEI',
 'ZEUS',
 'FONR',
 'DAL',
 'FRO',
 'FLXS',
 'EGY',
 'IVAC',
 'ECPG',
 'BH',
 'EGO',
 'PDS',
 'NSSC',
 'WWW',
 'TCOM',
 'EVC',
 'PEGA',
 'PDFS',
 'VIRC',
 'AMD',
 'LMAT',
 'VICR',
 'ULBI',
 'AVXL',
 'LMNR',
 'MGM',
 'ZION',
 'SWBI',
 'VGR',
 'MRCY',
 'ALKS',
 'GFI',
 'FBP',
 'BELFA',
 'EXTR',
 'WERN',
 'ICLR',
 'XPO',
 'AMWD',
 'MDC',
 'QDEL',
 'OLED',
 'PCYO',
 'ATGE',
 'GPRE',
 'DGII',
 'CVCO',
 'CTRA',
 'VTRS',
 'RF',
 'DHI',
 'EBIX',
 'TR',
 

In [17]:
with open('final_selection','rb') as f:
     final_selection = pickle.load(f)
selection = final_selection["finer"].union(final_selection["b"]).union(final_selection["c"]).union(final_selection["d"]).union(final_selection["finer finer pick"])
     

In [9]:
final_pick = []
final_b = []
final_c = []
final_d = []
final_final_pick = []
for i in pickle_contents:
    for j in range(len(i)):
        if i[j][0]["ticker"] in selection:
            stock = algo2_test(**i[j][0])
            # eval(stock, opt=False)
            maximum_drawdown, ROI, win_loss_ratio = eval(stock, ticker=i[j][0]["ticker"], variables=i[j][0],opt=True)
            buy_orders = []
            sell_orders = []
            current_order = None
            money = 100000
            short = None
            long = None

            for index, row in stock.iterrows():
                if row['order'] > 0:
                    if current_order is None and short is None:
                        current_order = {"Initial Money": money, 'Buy Date': row['date'], 'Buy Price': stock.loc[index -
                                                                                                                1, 'close'], "Shares Purchased": money/stock.loc[index - 1, 'close'], "long/short": "long"}
                        long = True
                    elif current_order is not None and short is not None:
                        current_order['Sell Date'] = row['date']
                        current_order['Sell Price'] = row['close']
                        current_order["Price Change"] = current_order["Sell Price"] - \
                            current_order["Buy Price"]
                        current_order["Profit"] = current_order["Price Change"] * \
                            current_order["Shares Purchased"] * -1
                        current_order["Money After Order"] = current_order["Shares Purchased"] * \
                            current_order['Buy Price'] + current_order["Profit"]
                        money = current_order["Money After Order"]
                        sell_orders.append(current_order)
                        short = None
                        current_order = {"Initial Money": money, 'Buy Date': row['date'], 'Buy Price': row[
                            'close'], "Shares Purchased": money/row['close'], "long/short": "long"}
                        long = True
                    else:
                        continue
                elif row['order'] < 0:
                    if current_order is None and long is None:
                        current_order = {"Initial Money": money, 'Buy Date': row['date'], 'Buy Price': stock.loc[index -
                                                                                                                1, 'close'], "Shares Purchased": money/stock.loc[index - 1, 'close'], "long/short": "short"}
                        short = True
                    elif current_order is not None and long is not None:
                        current_order['Sell Date'] = row['date']
                        current_order['Sell Price'] = row['close']
                        current_order["Price Change"] = current_order["Sell Price"] - \
                            current_order["Buy Price"]
                        current_order["Profit"] = current_order["Price Change"] * \
                            current_order["Shares Purchased"]
                        current_order["Money After Order"] = current_order["Shares Purchased"] * \
                            current_order['Buy Price'] + current_order["Profit"]
                        money = current_order["Money After Order"]
                        sell_orders.append(current_order)
                        long = None
                        current_order = {"Initial Money": money, 'Buy Date': row['date'], 'Buy Price': row[
                            'close'], "Shares Purchased": money/row['close'], "long/short": "short"}
                        short = True
                    else:
                        continue
                elif row['order'] == 0:
                    if current_order is not None:
                        current_order['Sell Date'] = row['date']
                        current_order['Sell Price'] = row['close']
                        current_order["Price Change"] = current_order["Sell Price"] - \
                            current_order["Buy Price"]
                        if current_order['long/short'] == "short":
                            current_order["Profit"] = current_order["Price Change"] * \
                                current_order["Shares Purchased"] * -1
                        elif current_order['long/short'] == "long":
                            current_order["Profit"] = current_order["Price Change"] * \
                                current_order["Shares Purchased"]
                        current_order["Money After Order"] = current_order["Shares Purchased"] * \
                            current_order['Buy Price'] + current_order["Profit"]
                        money = current_order["Money After Order"]
                        sell_orders.append(current_order)
                        current_order = None
                        long = None
                        short = None

            # Create a new DataFrame for buy and sell orders
            orders_df = pd.DataFrame(buy_orders + sell_orders)

            if ROI >= 2 and maximum_drawdown <= 0.12 and win_loss_ratio >= 0.75:
                if i[j][0]["ticker"] not in final_pick:
                    final_pick.append(i[j][0]["ticker"])
            if ROI >= 2 and maximum_drawdown <= 0.10 and win_loss_ratio >= 0.78:
                if i[j][0]["ticker"] not in final_b:
                    final_b.append(i[j][0]["ticker"])
            if ROI >= 2 and maximum_drawdown <= 0.15 and win_loss_ratio >= 0.8:
                if i[j][0]["ticker"] not in final_c:
                    final_c.append(i[j][0]["ticker"])
            if ROI >= 3 and maximum_drawdown <= 0.15 and win_loss_ratio >= 0.72:
                if i[j][0]["ticker"] not in final_d:
                    final_d.append(i[j][0]["ticker"])
            if ROI >=5 or maximum_drawdown <= 0.06 or win_loss_ratio >= 0.85:
                if i[j][0]["ticker"] not in final_final_pick:
                    final_final_pick.append(i[j][0]["ticker"])

In [22]:
final_select = set(final_pick).union(set(final_b)).union(set(final_c)).union(set(final_d)).union(set(final_final_pick))
len(final_select)

78

In [23]:
with open("final_pick", "wb") as fp:   #Pickling
    pickle.dump(final_select, fp)

In [24]:
final_select

{'AAL',
 'ABG',
 'AGYS',
 'ALV',
 'AN',
 'APT',
 'AVAV',
 'AYI',
 'CTO',
 'CVCO',
 'CVE',
 'DALN',
 'DB',
 'EBIX',
 'EGO',
 'EPAC',
 'EVO',
 'EZPW',
 'FHI',
 'FITB',
 'FONR',
 'FRPH',
 'FSTR',
 'GGAL',
 'GPRE',
 'HA',
 'HELE',
 'HUBG',
 'HZO',
 'IBN',
 'IESC',
 'ITGR',
 'KVHI',
 'LCII',
 'LPSN',
 'MDGL',
 'MET',
 'MGM',
 'MITK',
 'MRCY',
 'MSTR',
 'MT',
 'MTB',
 'OLED',
 'OSPN',
 'PAG',
 'PCYO',
 'PDFS',
 'PEGA',
 'PFIN',
 'PHM',
 'REX',
 'SAP',
 'SCX',
 'SHO',
 'SMFG',
 'SOHU',
 'SPXC',
 'SRCL',
 'SYNA',
 'TCOM',
 'TCX',
 'TNK',
 'TR',
 'TRC',
 'TSRI',
 'TYG',
 'UFPI',
 'ULBI',
 'UMC',
 'UNFI',
 'USPH',
 'UTMD',
 'VGR',
 'VIRC',
 'WERN',
 'XPO',
 'ZION'}

In [25]:
with open('final_select.txt', 'w') as file:
    for item in final_select:
        # Write each item on a new line
        file.write(f"{item}\n")